In [1]:
import torch
import torch.nn as nn
from dataloaders.train_dataloader import CustomDataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np


In [2]:

class AE(nn.Module):
    def __init__(self, height=32, width=32, channels=3):
        super(AE, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(channels, 16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(16, 8, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(8, 4, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv2d(4, 4, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(4, 8, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(8, 16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(16, channels, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [3]:
train_dataset = CustomDataset(r'D:\yousef\SBME 4\1\DL\project\train_thumbnails')
train_loader = DataLoader(train_dataset, batch_size=32)


In [4]:
len(train_loader)

17

In [14]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

# Set device

# Hyperparameters
learning_rate = 0.001
num_epochs = 30

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

channels =  3
model = AE(channels).to(device)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    for i, data in tqdm(enumerate(train_loader, 0)):
        inputs= data
        inputs = inputs.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, inputs)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Print average loss for the epoch
    average_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss:.4f}")

print("Training finished.")
torch.save(model.state_dict(), 'autoencoder_model.pth')


0it [00:00, ?it/s]

17it [00:54,  3.21s/it]


Epoch [1/30], Loss: 3.1609


17it [00:53,  3.15s/it]


Epoch [2/30], Loss: 2.8326


6it [00:21,  3.61s/it]


KeyboardInterrupt: 

In [5]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")


device(type='cuda')

In [5]:
torch.__version__

'2.1.1+cu118'

In [13]:
torch.cuda.device_count()

1

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
channels = 3
model = AE(channels)
model.load_state_dict(torch.load('autoencoder_model.pth'))

<All keys matched successfully>

In [6]:
class EncoderModel(nn.Module):
    def __init__(self, encoder):
        super(EncoderModel, self).__init__()
        self.encoder = encoder

    def forward(self, x):
        x = self.encoder(x)
        return x

In [7]:
encoder = EncoderModel(model.encoder.eval())

In [10]:
test_loader = DataLoader(train_dataset, batch_size=1)
features = []
for i, data in tqdm(enumerate(test_loader, 0)):
    inputs = data.to(device)
    feature = encoder(inputs)
    features.append(feature)

features


513it [01:04,  7.94it/s]


[tensor([[[[ 6.9655,  4.5212,  4.7262,  ...,  4.4657,  4.4657,  4.4657],
           [ 6.9655,  6.0529, 14.3486,  ..., 16.2750,  5.3178,  4.4657],
           [ 6.9655,  6.1804,  9.9207,  ..., 17.9900, 16.8472,  4.9407],
           ...,
           [ 6.1140, 12.7978,  0.1740,  ...,  4.4657,  4.4657,  4.4657],
           [ 8.7726,  8.7724,  0.0000,  ...,  4.4657,  4.4657,  4.4657],
           [ 7.2301,  6.1750,  4.0788,  ...,  5.1384,  5.1384,  5.1384]],
 
          [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.5922],
           [ 0.0000,  0.0000, 19.5263,  ..., 13.3268,  0.7364,  0.5922],
           [ 0.0000,  2.7542, 44.5221,  ..., 39.8319, 15.2535,  0.5922],
           ...,
           [13.3006, 42.1919, 49.5079,  ...,  0.0000,  0.0000,  0.5922],
           [ 0.0000, 33.5278, 43.8052,  ...,  0.0000,  0.0000,  0.5922],
           [ 0.0000,  3.4981, 12.0478,  ...,  0.0000,  0.0000,  0.6281]],
 
          [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 

In [11]:
f = torch.cat(features, dim=0)


In [13]:
f = f.detach().cpu().numpy()

In [15]:
f.shape

(513, 4, 16, 16)

In [16]:
f = f.reshape((513, 4* 16* 16))

In [17]:
f.shape

(513, 1024)

In [18]:
from sklearn import svm

nus = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
models = list(range(len(nus)))
p = list(range(len(nus)))
for i, nu in enumerate(nus):
    # train with nu
    models[i] = svm.OneClassSVM(nu=nu, kernel='rbf')
    models[i].fit(f)
    models[i].predict(f)
    p[i] = models[i].predict(f)


In [19]:
orig_data = np.zeros(513)

In [20]:
for pred in p:
    print(len(orig_data[pred == -1]))

52
108
155
207
257
308
359
409
462


In [37]:
for pred in p:
    print(len(orig_data[pred == -1]))

52
104
155
205
256
310
360
409
462


In [21]:
s = svm.OneClassSVM(nu=0.1, kernel='poly')
s.fit(f)
z = s.predict(f)
print(len(orig_data[z == -1]))

51
